#### **DATASET**

Antes de usar el dataset realizamos una análisis exploratorio de los datos para verificar si no hay datos faltantes o datos duplicados.

In [ ]:
import ollama
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt

from umap import UMAP
from ast import literal_eval
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA


In [ ]:
# Carga del dataset
df = pd.read_csv("data/hotel_reviews.csv")

In [ ]:
# Visualizamos los primero 5 registros
df.head(5)

In [ ]:
# Extraemos en otro dataset las columnas que nos interesan
df = df[["name", "description", "rating"]]

# Visualizamos nuevamente los primero 5 registros del dataset con los datos de interes
df.head(5)

In [ ]:
# Validamos si tenemos datos faltantes
print(df.isna().sum())

#### **EMBEDDINGS**

Haciendo uso del modelo de embeddings ```nomic-embed-text``` en su versión 1.5 creamos los embeddings

In [ ]:
embedding_model = "nomic-embed-text:latest"

# Función para obtener el embedding de un texto, la cual tambien reemplaza los saltos de linea del textp por espacios
def get_embedding(text, model=embedding_model):
  text = text.replace("\n", " ")
  return ollama.embeddings(model=model, prompt=text)["embedding"]

In [ ]:
# Generación de embeddings en una nueva columna del dataset
df['embedding'] = df.description.apply(lambda x: get_embedding(text=x, model=embedding_model))

In [ ]:
# Exportamos el dataset a trabajar a un archivo .csv
df.to_csv('data/hotel_reviews_embeddings.csv', index=False)

##### **Carga datos procesados**

In [ ]:
# Carga del dataset con los embeddings
df = pd.read_csv("data/hotel_reviews_embeddings.csv")

# Visualizamos los primeros 5 registros
df.head(5)

In [ ]:
# Se genera una matriz con los embeddings, de dimensiones (n_muestras, n_embedding)
matrix = np.array(df.embedding.apply(literal_eval).to_list())

In [ ]:
# Visualizamos las dimensiones de nuestra matriz
# Tenemos un dataset con 1264 muestras y cada muestra tiene un embedding de 768 dimensiones
print(matrix.shape)

#### **PCA**

##### **2 Componentes**

In [ ]:
# Mediante la tecnica de PCA vamos a pasar de 768 dimensiones a 2 dimensiones para visualizar los datos en un plano en 2 dimensiones
# y determinar que tanta información se pierde al reducir las dimensiones
pca = PCA(n_components=2)
componentes = pca.fit_transform(matrix)

# Graficamos los datos en 2 dimensiones
fig = px.scatter (
  componentes,
  x=0, y=1,
  color=df.rating.values-1,
  color_continuous_scale=px.colors.qualitative.Prism,
  title=f"Varianza total explicada: {pca.explained_variance_ratio_.sum() * 100:.2f}%",
  labels={'0': 'PC 1', '1': 'PC 2'}
)
fig.show()

Con la ```varianza total explicada``` para 2 componentes perdemos mas 90% de información

##### **3 Componentes**

In [ ]:
pca = PCA(n_components=3)
componentes = pca.fit_transform(matrix)

# Graficamos los datos en 3 dimensiones
fig = px.scatter_3d (
  componentes,
  x=0, y=1, z=2,
  color=df.rating.values-1,
  color_continuous_scale=px.colors.qualitative.Prism,
  title=f"Varianza total explicada: {pca.explained_variance_ratio_.sum() * 100:.2f}%",
  labels={'0': 'PC 1', '1': 'PC 2', '2': 'PC 3'}
)
fig.show()

Con la ```varianza total explicada``` para 3 componentes perdemos mas 85% de información

##### **Gráfico de Área**

A continuación mediante un gráfico de área trataremos de determinar la cantidad de componentes que nos permitan conservar cierto % de información donde consideremos el punto ideal (cantidad de componentes) de acuerdo a nuestra problemática.

In [ ]:
pca = PCA()
componentes = pca.fit_transform(matrix)
varianza_acumulativa = np.cumsum(pca.explained_variance_ratio_)

# Graficamos la varianza acumulativa explicada por cada componente
fig = px.area (
  componentes,
  x=range(1, varianza_acumulativa.shape[0] + 1),
  y=varianza_acumulativa,
  labels={"x": "Componentes", "y": "Varianza acumulativa"},
  title="Varianza acumultiva por componente"
)
fig.show()

Podemos deducir que aproximadamente en 190 componentes podemos explicar el 90% de los datos, lo que significa una perdida aproximada del 10% de los datos.

#### **t-SNE**

In [ ]:
tsne = TSNE(n_components=2, random_state=42)
X_tsne = tsne.fit_transform(matrix)

print("KL Value: ", tsne.kl_divergence_)

fig = px.scatter(x=X_tsne[:, 0], y=X_tsne[:, 1], color=df.rating.values-1, color_continuous_scale=px.colors.qualitative.Prism)
fig.update_layout(title="TSNE visualización de embeddings", xaxis_title="TSNE 1", yaxis_title="TSNE 2")
fig.show()

En este primer gráfico con parámetros por defecto no es posible realizar alguna distinción de los grupos de acuerdo al rating

In [ ]:
perplexity = np.arange(5,50, 5)
divergence = []

for i in perplexity:
  tsne = TSNE(n_components=2, init="pca", perplexity=i)
  X_tsne = tsne.fit_transform(matrix)
  divergence.append(tsne.kl_divergence_)

fig = px.line(x=perplexity, y=divergence, markers=True)
fig.update_layout(xaxis_title="Valores de perplejidad", yaxis_title="Divergencia")
fig.update_traces(line_color="blue", line_width=2)
fig.show()

In [ ]:
tsne = TSNE(n_components=2, perplexity=50, random_state=42)
X_tsne = tsne.fit_transform(matrix)

print("KL Value: ", tsne.kl_divergence_)

fig = px.scatter(x=X_tsne[:, 0], y=X_tsne[:, 1], color=df.rating.values-1, color_continuous_scale=px.colors.qualitative.Prism)
fig.update_layout(title="TSNE visualización de embeddings", xaxis_title="TSNE 1", yaxis_title="TSNE 2")
fig.show()

#### **UMAP**

In [ ]:
umap = UMAP(n_components=2, random_state=42)
X_umap = umap.fit_transform(matrix)

fig = px.scatter(x=X_umap[:, 0], y=X_umap[:, 1], color=df.rating.values-1, color_continuous_scale=px.colors.qualitative.Prism)
fig.update_layout(
    title="UMAP visualizacion de embeddings",
    xaxis_title="First UMAP",
    yaxis_title="Second UMAP",
)
fig.show()

In [ ]:
n_neighbors = np.arange(5, 55, 5)

for i in n_neighbors:
  umap = UMAP(n_components=2, n_neighbors=i, random_state=42)
  X_umap = umap.fit_transform(matrix)
  
  fig = px.scatter(x=X_umap[:, 0], y=X_umap[:, 1], color=df.rating.values-1, color_continuous_scale=px.colors.qualitative.Prism)
  fig.update_layout(
    title=f"UMAP visualizacion con vecinos = {i}",
    xaxis_title="First UMAP component",
    yaxis_title="Second UMAP component"
  )
  fig.show()

In [ ]:
min_dist_values = np.arange(0.0, 1.0, 0.1)

for min_dist in min_dist_values:
  umap = UMAP(n_components=2, min_dist=min_dist, random_state=42)
  X_umap = umap.fit_transform(matrix)

  fig = px.scatter(x=X_umap[:, 0], y=X_umap[:, 1], color=df.rating.values-1, color_continuous_scale=px.colors.qualitative.Prism)
  fig.update_layout(
    title=f"UMAP visualizacion con distancia = {min_dist:.1f}",
    xaxis_title="First UMAP component",
    yaxis_title="Second UMAP component"
  )
  fig.show()